In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths to training and validation data
data_dir = "D:/Major Project/normalized"
train_dir = os.path.join(data_dir, "train")
valid_dir = os.path.join(data_dir, "valid")

# ImageDataGenerators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(rescale=1./255)

# Data generators (Binary Classification)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'  # Only two classes: cavity and implant
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Inspect class indices
print("Class indices:", train_generator.class_indices)

# Inspect sample batch
x_batch, y_batch = next(train_generator)
print("Sample image batch shape:", x_batch.shape)
print("Sample label batch shape:", y_batch.shape)

# Define the CNN model (Binary Classification)
model = Sequential()
model.add(Input(shape=(224, 224, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(160, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))  # Single output for binary classification

# Compile the model with binary crossentropy
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the model
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=30,
    callbacks=[early_stop, lr_scheduler]
)

# Save the trained model
model.save("models/cnn_teeth_health_binary.h5")


Found 13123 images belonging to 4 classes.
Found 4121 images belonging to 4 classes.
Class indices: {'cavity': 0, 'fillings': 1, 'impacted tooth': 2, 'implant': 3}
Sample image batch shape: (32, 224, 224, 3)
Sample label batch shape: (32,)
Epoch 1/30
411/411 ━━━━━━━━━━━━━━━━━━━━ 436s 1s/step - accuracy: 0.2488 - loss: -4245.8169 - val_accuracy: 0.2570 - val_loss: -24203.2324 - learning_rate: 1.0000e-04
Epoch 2/30
411/411 ━━━━━━━━━━━━━━━━━━━━ 423s 1s/step - accuracy: 0.2620 - loss: -140027.1094 - val_accuracy: 0.2570 - val_loss: -720253.5000 - learning_rate: 1.0000e-04
Epoch 3/30
411/411 ━━━━━━━━━━━━━━━━━━━━ 423s 1s/step - accuracy: 0.2547 - loss: -842048.3750 - val_accuracy: 0.2570 - val_loss: -1760519.2500 - learning_rate: 1.0000e-04
Epoch 4/30
411/411 ━━━━━━━━━━━━━━━━━━━━ 425s 1s/step - accuracy: 0.2538 - loss: -2570808.5000 - val_accuracy: 0.2570 - val_loss: -4352746.5000 - learning_rate: 1.0000e-04
Epoch 5/30
411/411 ━━━━━━━━━━━━━━━━━━━━ 425s 1s/step - accuracy: 0.2520 - loss: -607